In [1]:
# !https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
# !multi_cased_L-12_H-768_A-12.zip
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news-30k/news-30k.json.zip
# !unzip news-30k.json.zip

In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
from tensor2tensor.utils import beam_search, rouge

/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-pa

In [3]:
import json
from unidecode import unidecode
import re

In [4]:
with open('news-30k.json') as fopen:
    news = json.load(fopen)
len(news)

29855

In [5]:
import malaya
import re
tokenizer = malaya.preprocessing.SocialTokenizer().tokenize

accept_tokens = ',-.()"\''

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def detect_money(word):
    if word[:2] == 'rm' and is_number_regex(word[2:]):
        return True
    else:
        return False

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = [w.lower() for w in tokenized if len(w) > 1 or w in accept_tokens]
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    tokenized = ['<MONEY>' if detect_money(w) else w for w in tokenized]
    return tokenized

def clean_label(label):
    string = re.sub('[^A-Za-z\- ]+', ' ', label)
    return re.sub(r'[ ]+', ' ', string.lower()).strip()

<>:8: DeprecationWarning: invalid escape sequence \d
<>:26: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \d
<>:26: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \d
<>:26: DeprecationWarning: invalid escape sequence \-
<ipython-input-5-85c9669409f4>:8: DeprecationWarning: invalid escape sequence \d
  if re.match("^\d+?\.\d+?$", s) is None:
<ipython-input-5-85c9669409f4>:26: DeprecationWarning: invalid escape sequence \-
  string = re.sub('[^A-Za-z\- ]+', ' ', label)


In [6]:
BERT_VOCAB = 'multi_cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'multi_cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'multi_cased_L-12_H-768_A-12/bert_config.json'

In [7]:
from tqdm import tqdm

min_len = 5
max_len = 400

x, y = [], []
for n in tqdm(news):
    if len(n['text'].split()) > min_len:
        p = preprocessing(n['text'])[:max_len]
        x.append(p)
        p = preprocessing(n['title'])
        y.append(p)

100%|██████████| 29855/29855 [01:04<00:00, 459.75it/s]


In [8]:
import collections
import itertools

def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [9]:
import itertools

concat = list(itertools.chain(*x))
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])
print('filtered vocab size:',len(dictionary))
print("% of vocab used: {}%".format(round(len(dictionary)/vocabulary_size,4)*100))

vocab from size: 85652
Most common words [(',', 362826), ('.', 320679), ('yang', 149889), ('dan', 139864), ('di', 119719), ('-', 112657)]
Sample data [4340, 288, 1409, 340, 1580, 110, 3480, 4, 10, 4] ['waris', 'keluarga', 'allahyarham', 'muhammad', 'haziq', 'mohd', 'tarmizi', ',', '<NUM>', ',']
filtered vocab size: 85656
% of vocab used: 100.0%


In [10]:
MAX_SEQ_LENGTH = 400

tokenization.validate_case_matches_checkpoint(False,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=False)

In [11]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(x):
    tokens_a = tokenizer.tokenize(' '.join(text))
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 29855/29855 [02:31<00:00, 197.43it/s]


In [12]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [13]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [14]:
for i in range(len(y)):
    y[i].append('EOS')

In [15]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
epoch = 20
batch_size = 5
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [16]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate)
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 to_dict_size, learning_rate, kernel_size = 2, n_attn_heads = 16):
        
        self.X = tf.placeholder(tf.int32, [None, max_len])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.batch_size = batch_size
        
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        self.num_layers = num_layers
        self.kernel_size = kernel_size
        self.size_layer = size_layer
        self.n_attn_heads = n_attn_heads
        self.dict_size = to_dict_size
        
        self.training_logits, coverage_loss = self.forward(self.X, decoder_input)
        maxlen = tf.reduce_max(self.Y_seq_len)
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        
        targets = tf.slice(self.Y, [0, 0], [-1, maxlen])
        i1, i2 = tf.meshgrid(tf.range(batch_size),
                     tf.range(maxlen), indexing="ij")
        indices = tf.stack((i1,i2,targets),axis=2)
        probs = tf.gather_nd(self.training_logits, indices)
        probs = tf.where(tf.less_equal(probs,0),tf.ones_like(probs)*1e-10,probs)
        crossent = -tf.log(probs)
        self.cost = tf.reduce_sum(crossent * masks) / tf.to_float(batch_size)
        self.coverage_loss = tf.reduce_sum(coverage_loss / tf.to_float(batch_size))
        self.cost = self.cost + self.coverage_loss
    
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def _calc_final_dist(self, x, gens, vocab_dists, attn_dists):
        with tf.variable_scope('final_distribution', reuse=tf.AUTO_REUSE):
            vocab_dists = gens * vocab_dists
            attn_dists = (1-gens) * attn_dists
            batch_size = tf.shape(attn_dists)[0]
            dec_t = tf.shape(attn_dists)[1]
            attn_len = tf.shape(attn_dists)[2]

            dec = tf.range(0, limit=dec_t) # [dec]
            dec = tf.expand_dims(dec, axis=-1) # [dec, 1]
            dec = tf.tile(dec, [1, attn_len]) # [dec, atten_len]
            dec = tf.expand_dims(dec, axis=0) # [1, dec, atten_len]
            dec = tf.tile(dec, [batch_size, 1, 1]) # [batch_size, dec, atten_len]

            x = tf.expand_dims(x, axis=1) # [batch_size, 1, atten_len]
            x = tf.tile(x, [1, dec_t, 1]) # [batch_size, dec, atten_len]
            x = tf.stack([dec, x], axis=3)

            attn_dists_projected = tf.map_fn(fn=lambda y: \
                                            tf.scatter_nd(y[0], y[1], [dec_t, self.dict_size]),
                                            elems=(x, attn_dists), dtype=tf.float32)

            final_dists = attn_dists_projected + vocab_dists
            return final_dists
        
    def forward(self, x, y, reuse = False):
        with tf.variable_scope('forward',reuse=reuse):
            with tf.variable_scope('forward',reuse=reuse):
                model = modeling.BertModel(
                config=bert_config,
                is_training=True,
                input_ids=x,
                use_one_hot_embeddings=False)
                encoder_embedded = model.get_sequence_output()
                decoder_embedded = tf.nn.embedding_lookup(self.embedding, y)
                decoder_embedded += position_encoding(decoder_embedded)
                
                g = tf.identity(decoder_embedded)
                dec = decoder_embedded
                attn_dists = []
                for i in range(self.num_layers):
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate
                    with tf.variable_scope('decode_%d'%i,reuse=reuse):
                        attn_res = h = cnn_block(dec, dilation_rate, 
                                                 pad_sz, self.size_layer, self.kernel_size)
                        C = []
                        for j in range(self.n_attn_heads):
                            h_ = tf.layers.dense(h, self.size_layer//self.n_attn_heads)
                            g_ = tf.layers.dense(g, self.size_layer//self.n_attn_heads)
                            zu_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)
                            ze_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)

                            d = tf.layers.dense(h_, self.size_layer//self.n_attn_heads) + g_
                            dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                            a = tf.nn.softmax(dz)
                            attn_dists.append(a)
                            c_ = tf.matmul(a, ze_)
                            C.append(c_)

                        c = tf.concat(C, 2)
                        h = tf.layers.dense(attn_res + c, self.size_layer)
                        dec += h

                weights = tf.transpose(self.embedding)
                logits = tf.einsum('ntd,dk->ntk', dec, weights)
                print(decoder_embedded, dec, attn_dists[-1])

                with tf.variable_scope("gen", reuse=tf.AUTO_REUSE):
                    gens = tf.layers.dense(tf.concat([decoder_embedded, dec, attn_dists[-1]], axis=-1), 
                                               units=1, activation=tf.sigmoid, use_bias=False)
                    
                logits = tf.nn.softmax(logits)
                
                print(gens)
                alignment_history = tf.transpose(attn_dists[-1],[1,2,0])
                coverage_loss = tf.minimum(alignment_history,tf.cumsum(alignment_history, axis=2, exclusive=True))
                        
                return self._calc_final_dist(x, gens, logits, attn_dists[-1]), coverage_loss

In [17]:
size_layer = 256
num_layers = 4
embedded_size = 256
learning_rate = 2e-5

In [18]:
def beam_search_decoding(length = 20, beam_width = 5):
    initial_ids = tf.fill([model.batch_size], GO)
    
    def symbols_to_logits(ids):
        x = tf.contrib.seq2seq.tile_batch(model.X, beam_width)
        logits, _ = model.forward(x, ids, reuse = True)
        return logits[:, tf.shape(ids)[1]-1, :]

    final_ids, final_probs, _ = beam_search.beam_search(
        symbols_to_logits,
        initial_ids,
        beam_width,
        length,
        len(dictionary),
        0.0,
        eos_id = EOS)
    
    return final_ids

In [19]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, 
                len(dictionary), learning_rate)
model.generate = beam_search_decoding()
sess.run(tf.global_variables_initializer())

Tensor("forward/forward/add:0", shape=(?, ?, 256), dtype=float32) Tensor("forward/forward/decode_3/add_19:0", shape=(?, ?, 256), dtype=float32) Tensor("forward/forward/decode_3/Softmax_15:0", shape=(?, ?, 400), dtype=float32)
Tensor("forward/forward/gen/dense/Sigmoid:0", shape=(?, ?, 1), dtype=float32)
Tensor("while/forward/forward/add:0", shape=(?, ?, 256), dtype=float32) Tensor("while/forward/forward/decode_3/add_19:0", shape=(?, ?, 256), dtype=float32) Tensor("while/forward/forward/decode_3/Softmax_15:0", shape=(?, ?, 400), dtype=float32)
Tensor("while/forward/forward/gen/dense/Sigmoid:0", shape=(?, ?, 1), dtype=float32)


In [20]:
def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if 'bert' not in name:
            continue
        assignment_map[name] = name_to_variable['forward/forward/'+name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [21]:
tvars = tf.trainable_variables()
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                BERT_INIT_CHKPNT)

In [22]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, BERT_INIT_CHKPNT)

INFO:tensorflow:Restoring parameters from multi_cased_L-12_H-768_A-12/bert_model.ckpt


In [23]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i:
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [24]:
Y = str_idx(y, dictionary)

In [25]:
from sklearn.cross_validation import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(input_ids, Y, test_size = 0.1)

In [26]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [27]:
sess.run(model.generate, feed_dict = {model.X: [test_X[0]]})

array([[[    1, 36811, 36811, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635],
        [    1, 36811, 36811, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635, 75635, 38993, 70602, 70602, 70602, 70602,
         70602, 70602, 70602],
        [    1, 36811, 36811, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635, 38541, 38541, 38541, 38541, 38541, 38541,
         38541, 38541, 38541],
        [    1, 36811, 36811, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 48340],
        [    1, 36811, 36811, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635, 75635,
         75635, 75635, 47233]]], dtype=int32)

In [28]:
from keras.preprocessing import sequence

def calculate_rouges(predicted, batch_y):
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False)
    batch_y = np.array(batch_y)
    rouges = []
    for i in range(predicted.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        rouges.append(rouge.rouge_n([p], [a]))
    return np.mean(rouges)

Using TensorFlow backend.


In [29]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(20):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    rouge_train, rouge_test = 0, 0
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x = train_X[k: index]
        batch_y, _ = pad_sentence_batch(train_Y[k: index], PAD)
        l, acc, loss, _ = sess.run([model.training_logits, model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += acc
        r = calculate_rouges(np.argmax(l, axis = 2), batch_y)
        rouge_train += r
        pbar.set_postfix(cost=loss, accuracy = acc, rouge_2 = r)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x = test_X[k: index]
        batch_y, _ = pad_sentence_batch(test_Y[k: index], PAD)
        l, acc, loss = sess.run([model.training_logits, model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss_test += loss
        total_accuracy_test += acc
        r = calculate_rouges(np.argmax(l, axis = 2), batch_y)
        rouge_test += r
        pbar.set_postfix(cost=loss, accuracy = acc, rouge_2 = r)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
    rouge_train /= (len(train_X) / batch_size)
    rouge_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))
    print('epoch: %d, avg train rouge: %f, avg test rouge: %f'%(EPOCH, rouge_train, rouge_test))

train minibatch loop:   0%|          | 0/5374 [00:00<?, ?it/s]

epoch: 0, avg loss: 279.728556, avg accuracy: 0.046760
epoch: 0, avg loss test: 192.069527, avg accuracy test: 0.059605
epoch: 0, avg train rouge: 0.145015, avg test rouge: 0.062441


train minibatch loop:   0%|          | 0/5374 [00:00<?, ?it/s]

epoch: 1, avg loss: 159.724705, avg accuracy: 0.077480
epoch: 1, avg loss test: 136.929695, avg accuracy test: 0.085008
epoch: 1, avg train rouge: 0.035897, avg test rouge: 0.039846


train minibatch loop:   0%|          | 0/5374 [00:00<?, ?it/s]

...

train minibatch loop:  13%|█▎        | 693/5374 [16:24<1:50:43,  1.42s/it, accuracy=0.857, cost=8.29, rouge_2=0.687] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  73%|███████▎  | 3947/5374 [1:33:09<33:37,  1.41s/it, accuracy=0.763, cost=8.89, rouge_2=0.559]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:   0%|          | 0/5374 [00:00<?, ?it/s]

epoch: 16, avg loss: 10.116217, avg accuracy: 0.762534
epoch: 16, avg loss test: 77.958384, avg accuracy test: 0.355144
epoch: 16, avg train rouge: 0.599804, avg test rouge: 0.222278


train minibatch loop:  25%|██▍       | 1341/5374 [31:27<1:34:03,  1.40s/it, accuracy=0.795, cost=5.49, rouge_2=0.693]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  86%|████████▋ | 4639/5374 [1:49:05<17:23,  1.42s/it, accuracy=0.745, cost=10.8, rouge_2=0.495]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  38%|███▊      | 2058/5374 [48:36<1:18:27,  1.42s/it, accuracy=0.732, cost=9.74, rouge_2=0.502

epoch: 18, avg loss: 6.683461, avg accuracy: 0.824960
epoch: 18, avg loss test: 81.337019, avg accuracy test: 0.360266
epoch: 18, avg train rouge: 0.690060, avg test rouge: 0.230481


train minibatch loop:  81%|████████  | 4366/5374 [1:43:03<23:46,  1.41s/it, accuracy=0.94, cost=4.71, rouge_2=0.877]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
generated = [rev_dictionary[i] for i in sess.run(model.generate, feed_dict = {model.X: [test_X[0]]})[0,0,:]]
' '.join(generated)

'GO tempatan EOS PAD'

In [31]:
' '.join([rev_dictionary[i] for i in test_Y[0]])

'pas biadab kata anwar bekas banduan EOS'

In [32]:
rouge_test = 0
total_loss_test, total_accuracy_test = 0, 0
pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
for k in pbar:
    index = min(k+batch_size,len(test_X))
    batch_x = test_X[k: index]
    batch_y, _ = pad_sentence_batch(test_Y[k: index], PAD)
    l, acc, loss = sess.run([model.training_logits, model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
    total_loss_test += loss
    total_accuracy_test += acc
    r = calculate_rouges(np.argmax(l, axis = 2), batch_y)
    rouge_test += r
    pbar.set_postfix(cost=loss, accuracy = acc, rouge_2 = r)

total_loss_test /= (len(test_X) / batch_size)
total_accuracy_test /= (len(test_X) / batch_size)
rouge_test /= (len(test_X) / batch_size)
print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))
print('epoch: %d, avg test rouge: %f'%(EPOCH, rouge_test))

test minibatch loop: 100%|██████████| 598/598 [04:57<00:00,  2.51it/s, accuracy=1, cost=1.18, rouge_2=1]         

epoch: 19, avg loss test: 83.170347, avg accuracy test: 0.371790
epoch: 19, avg test rouge: 0.237134
